In [1]:
from google.colab import files
uploaded = files.upload()

Saving Buah_Klasifikasi (2).zip to Buah_Klasifikasi (2).zip


In [2]:
import zipfile
import os

# Nama file ZIP yang diunggah
zip_file_name = "Buah_Klasifikasi (2).zip"  # Ganti dengan nama file ZIP Anda

# Ekstraksi file ZIP
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("dataset")  # Ekstraksi ke folder "dataset"

# Cek apakah dataset berhasil diekstrak
print("Isi folder dataset:", os.listdir("dataset"))

Isi folder dataset: ['Buah_Klasifikasi']


In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [4]:
import cv2
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import gradio as gr

# 1. Fungsi untuk memuat dan mengubah ukuran citra
def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    image = cv2.resize(image, (128, 128))  # Ubah ukuran gambar
    image = image / 255.0  # Normalisasi gambar
    return image

# 2. Fungsi untuk memuat dataset citra
def load_dataset(dataset_dir):
    images = []
    labels = []

    for label in range(1, 6):  # Terdapat 5 tingkatan kematangan buah
        label_dir = os.path.join(dataset_dir, f'tingkat{label}')
        if not os.path.exists(label_dir):
            print(f"Peringatan: Direktori tidak ditemukan: {label_dir}")
            continue

        for file_name in os.listdir(label_dir):
            image_path = os.path.join(label_dir, file_name)
            image = load_image(image_path)
            if image is None:
                print(f"Peringatan: Tidak dapat membaca gambar: {image_path}")
                continue

            images.append(image)
            # Mengurangi label dengan 1 agar dimulai dari 0 (untuk kompatibilitas dengan model)
            labels.append(label - 1)

    return np.array(images), np.array(labels)

# 3. Membagi dataset menjadi data latih dan data uji
def split_dataset(images, labels):
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# 4. Membangun model CNN untuk klasifikasi citra
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(5, activation='softmax')  # 5 kelas untuk tingkat kematangan (output kelas 0-4)
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 5. Melatih model
def train_model(X_train, y_train):
    model = build_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
    return model

# 6. Mengevaluasi model
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    return accuracy

# 7. Fungsi prediksi untuk Gradio
def predict_maturity(image_path):
    image = load_image(image_path)  # Baca gambar dari path
    if image is None:
        return "Error: Gambar tidak valid atau tidak dapat dibaca."

    image = np.expand_dims(image, axis=0)  # Tambahkan dimensi batch untuk prediksi
    predicted_maturity = model.predict(image)
    predicted_class = np.argmax(predicted_maturity, axis=1)[0]  # Kelas yang diprediksi (dimulai dari 0)
    predicted_class += 1  # Mengubah label kembali ke range asli (mulai dari 1)
    return f"Tingkat Kematangan Buah: {predicted_class}"

# 8. Memuat dataset
dataset_dir = '/content/dataset/Buah_Klasifikasi'  # Ganti dengan path dataset Anda
images, labels = load_dataset(dataset_dir)

# Memeriksa apakah dataset kosong
if len(images) == 0 or len(labels) == 0:
    raise ValueError(f"Error: Dataset kosong atau tidak valid. Periksa path dataset: {dataset_dir}")

# 9. Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = split_dataset(images, labels)

# 10. Melatih model
model = train_model(X_train, y_train)

# 11. Evaluasi model
accuracy = evaluate_model(model, X_test, y_test)
print(f"Akurasi model: {accuracy:.2f}")

# 12. Antarmuka Gradio
interface = gr.Interface(
    fn=predict_maturity,
    inputs=gr.Image(type="filepath", label="Unggah Gambar Buah"),
    outputs=gr.Textbox(label="Hasil Prediksi"),
    title="Klasifikasi Tingkat Kematangan Buah",
    description=f"Akurasi model: {accuracy:.2f}"
)

# 13. Menjalankan Gradio
interface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.1875 - loss: 1.6254 - val_accuracy: 0.1250 - val_loss: 2.8699
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2812 - loss: 2.5430 - val_accuracy: 0.2500 - val_loss: 1.6676
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2188 - loss: 1.8164 - val_accuracy: 0.2500 - val_loss: 1.6068
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2812 - loss: 1.6314 - val_accuracy: 0.1250 - val_loss: 1.7037
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3125 - loss: 1.5352 - val_accuracy: 0.1250 - val_loss: 1.9727
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2812 - loss: 1.5964 - val_accuracy: 0.1250 - val_loss: 1.8030
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 945ms/step - accuracy: 0.2812 - loss: 1.4588 - val_accuracy: 0.1250 - val_loss: 1.7768
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 940ms/step - accuracy: 0.5938 - loss: 1.3956 - val_accuracy: 0.1250 - val_loss: 1.9010
Epoch 9/10